<a href="https://colab.research.google.com/github/shnhrtkyk/JTCcode/blob/main/04_%E3%83%97%E3%83%AA%E3%83%9F%E3%83%86%E3%82%A3%E3%83%96%E6%A4%9C%E5%87%BA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# プリミティブ検出


点群のなかから、平面や球などの単純な図形を検出する処理を「プリミティブ検出」と呼びます。
机の上に並べた対象物を検出したい場合、事前に平面を検出しておき、その部分の点群を削除してしまえば、
個々の物体を簡単に単離（検出）することができます。

このように身の回りの構造物の多くは、単純な数式で記述できるプリミティブ形状であることが多いため、
プリミティブ検出は、シーン理解のためのかなり強力な前処理として利用することができます。
ここでは、RANSACを使ったプリミティブ検出アルゴリズムについて紹介します。

## データのダウンロード
机の上に日用品がおかれた状態を点群で観測した公開データを用います。

In [ ]:
!git clone --recursive https://github.com/3d-point-cloud-processing/3dpcp_book_codes

## ライブラリインストール

In [ ]:
!pip install open3d==0.16



## 平面の検出
Open3Dによる平面検出について紹介します。
対象となる点群データは```tabletop_scene1.ply```です。読み込んで表示してみましょう。
このデータは机の上に配置した球やうさぎなどを計測した点群です。


In [ ]:
import open3d as o3d
import numpy as np
import copy

In [ ]:
pcd = o3d.io.read_point_cloud("/content/3dpcp_book_codes/data/tabletop_scene.ply")
o3d.visualization.draw_plotly(
  [pcd],
  width=1200,
  height=800,
  front=[  -0.066170093019535442, 0.40212907450402163, -0.91318876812427185 ], # 表示位置の設定
  lookat=[ 0.02917175397384137, -0.0088117591406781711, 0.52859723520459312 ],   # 表示位置の設定
  up=[-0.032373407578752206, -0.91557794182184093, -0.40083537135714736]     # 表示位置の設定
)



この点群から、平面検出によって、机の面を検出してみましょう。
Open3Dでは、```segment_plane()```が平面検出を実行する関数として用意されています。
実行してみましょう。

In [ ]:
# Open3Dの平面検出の関数を利用する
plane_model, inliers = pcd.segment_plane(distance_threshold=0.005,
                                         ransac_n=3,
                                         num_iterations=500)
# 平面パラメータを確認する
[a, b, c, d] = plane_model
print(f"Plane equation: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0")

関数```segment_plane()```では、RANSACによる平面検出を実行します。
引数は次の通りです。
- distance_threshold
    - RANSACの「評価」処理で利用されます。平面のインライアとして判定するための距離のしきい値です。0.005を設定すると、平面から距離5mm以内の点をインライアとしてみなします。
- ransac_n
    - RANSACの「サンプリング」処理で利用されます。この点数から平面のパラメータを計算します。
- num_iteration
    - RANSACの「サンプリング」と「評価」の繰り返し回数です。

出力は次の通りです。
- plane_model
    - 平面パラメータ
- inliers
    - 元の点群における、平面上の点のインデックスのリスト
    

```inliers```には、平面上の点のインデックスが保存されていますので、これを使って結果を確認しましょう。
以下のコードでは、インデックスのリストを使って、点群を平面上のもの（赤色）と、それ以外に分けています。

In [ ]:
# 平面検出関数で取得したインデックスに基づいて、平面の点群を抽出する
plane_cloud = pcd.select_by_index(inliers)
# 平面の点群を赤色に着色する
plane_cloud.paint_uniform_color([1.0, 0, 0])
# 平面以外の点群を指定する
outlier_cloud = pcd.select_by_index(inliers, invert=True)

In [ ]:
o3d.visualization.draw_plotly(
  [plane_cloud, outlier_cloud],
  width=1200,
  height=800,
  front=[  -0.066170093019535442, 0.40212907450402163, -0.91318876812427185 ], # 表示位置の設定
  lookat=[ 0.02917175397384137, -0.0088117591406781711, 0.52859723520459312 ],   # 表示位置の設定
  up=[-0.032373407578752206, -0.91557794182184093, -0.40083537135714736]     # 表示位置の設定
)

机を計測した点群のみが赤色に変更されており、うまく平面検出に成功したことがわかります。

なお、平面検出の結果におけるplane_modelは平面を表現するパラメータです。
3次元の点を$(x,y,z)$とすると、平面は$ax+by+cz+d=0$で表せます。
したがって、平面検出をおこなうRANSACのモデルパラメータは$a,b,c,d$となります。

